# Predict students grades

Here [Student Performance Data Set](https://www.kaggle.com/larsen0966/student-performance-data-set) dataset by [Data-Science Sean](https://www.kaggle.com/larsen0966) is used to perform `EDA` and create a `machine learning model` that can predict student's final grades i.e. Tthe goal is to predict `G3` using `G1` and `G2`.

![](https://media.giphy.com/media/IPbS5R4fSUl5S/giphy.gif)

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.pipeline import Pipeline
from sklearn import linear_model, preprocessing
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import KFold, learning_curve, cross_val_score, train_test_split

from joblib import dump

In [ ]:
# Loading the dataset
df = pd.read_csv('/kaggle/input/student-performance-data-set/student-por.csv')
df.sample(5)

### 🧬 Attribute Information:

1. `school` - student's school (binary: 'GP' - Gabriel Pereira or 'MS' - Mousinho da Silveira)
2. `sex` - student's sex (binary: 'F' - female or 'M' - male)
3. `age` - student's age (numeric: from 15 to 22)
4. `address` - student's home address type (binary: 'U' - urban or 'R' - rural)
5. `famsize` - family size (binary: 'LE3' - less or equal to 3 or 'GT3' - greater than 3)
6. `Pstatus` - parent's cohabitation status (binary: 'T' - living together or 'A' - apart)
7. `Medu` - mother's education (numeric: 0 - none, 1 - primary education (4th grade), 2 â€“ 5th to 9th grade, 3 â€“ secondary education or 4 â€“ higher education)
8. `Fedu` - father's education (numeric: 0 - none, 1 - primary education (4th grade), 2 â€“ 5th to 9th grade, 3 â€“ secondary education or 4 â€“ higher education)
9. `Mjob` - mother's job (nominal: 'teacher', 'health' care related, civil 'services' (e.g. administrative or police), 'at_home' or 'other')
10. `Fjob` - father's job (nominal: 'teacher', 'health' care related, civil 'services' (e.g. administrative or police), 'at_home' or 'other')
11. `reason` - reason to choose this school (nominal: close to 'home', school 'reputation', 'course' preference or 'other')
12. `guardian` - student's guardian (nominal: 'mother', 'father' or 'other')
13. `traveltime` - home to school travel time (numeric: 1 - <15 min., 2 - 15 to 30 min., 3 - 30 min. to 1 hour, or 4 - >1 hour)
14. `studytime` - weekly study time (numeric: 1 - <2 hours, 2 - 2 to 5 hours, 3 - 5 to 10 hours, or 4 - >10 hours)
15. `failures` - number of past class failures (numeric: n if 1<=n<3, else 4)
16. `schoolsup` - extra educational support (binary: yes or no)
17. `famsup` - family educational support (binary: yes or no)
18. `paid` - extra paid classes within the course subject (Math or Portuguese) (binary: yes or no)
19. `activities` - extra-curricular activities (binary: yes or no)
20. `nursery` - attended nursery school (binary: yes or no)
21. `higher` - wants to take higher education (binary: yes or no)
22. `internet` - Internet access at home (binary: yes or no)
23. `romantic` - with a romantic relationship (binary: yes or no)
24. `famrel` - quality of family relationships (numeric: from 1 - very bad to 5 - excellent)
25. `freetime` - free time after school (numeric: from 1 - very low to 5 - very high)
26. `goout` - going out with friends (numeric: from 1 - very low to 5 - very high)
27. `Dalc` - workday alcohol consumption (numeric: from 1 - very low to 5 - very high)
28. `Walc` - weekend alcohol consumption (numeric: from 1 - very low to 5 - very high)
29. `health` - current health status (numeric: from 1 - very bad to 5 - very good)
30. `absences` - number of school absences (numeric: from 0 to 93)
31. `G1` - first period grade (numeric: from 0 to 20)
32. `G2` - second period grade (numeric: from 0 to 20)
33. `G3` - final grade (numeric: from 0 to 20, output target)

## ⛄️ Data preparation

In [ ]:
# Getting the average of the grades
df['Average Grades'] = df[['G1', 'G2', 'G3']].mean(axis='columns')
df.head()

In [ ]:
df.info()

No missing data

## 🏄‍♀️ Exploratory Data Analysis

**Helper plotting functions**

In [ ]:
def plot_corr(df, annot=True):
    _, ax = plt.subplots(figsize=(16, 12))
    sns.heatmap(
        df.corr(),
        annot=annot,
        cmap=sns.cubehelix_palette(start=.5, rot=-.5, as_cmap=True),
        ax=ax
    )


def plot_histplot(column):
    sns.histplot(x=column, color='#65b87b', alpha=.7) 
    
    
def plot_countplot(df, column_name, ax=None):
    _df = df[[column_name]].copy()
    if len(_df[_df[column_name].isnull()]):
        _df.fillna('NaN', inplace=True)
    
    color = '#42b0f5' if ax != None else '#7661ff'
    sns.countplot(x=column_name, data=_df, color=color, alpha=.7, ax=ax)
    del _df

In [ ]:
plot_histplot(df['Average Grades'])

**Pearson correlation coefficient significance** 
![Pearson correlation coefficient significance](https://miro.medium.com/max/466/1*Qz_gwy4ZaSZuOpl3IyO2HA.png)

In [ ]:
plot_corr(df)

> `age` has low positive correlation with `failure`
>
> `Medu` & `Fedu` has moderate positive correlation & they both have low positive correlation with `grades`
>
> `studytime` & `grades` have a low positive correlation
>
> `failure` has low negative correlation with `grades`
>
> `freetime` has low positive correlation with `goout`
>
> `goout` has low positive correlation with `Walc`
>
> `Walc` has moderate positive correlation with `Dalc` and they both have negligible negative correlation 

In [ ]:
def plot_base_relation(df, figsize=(20, 60)):
    columns = df.columns.tolist()
    _, axs = plt.subplots(len(columns), 3, figsize=figsize)
    
    for idx, column in enumerate(columns):
        # To get distribution of data
        sns.histplot(
            x=df[column],
            kde=False,
            color='#65b87b', alpha=.7,
            ax=axs[idx][0]
        )

        # To get knowledge about outliers
        sns.boxplot(
            x=df[column],
            color='#6fb9bd',
            ax=axs[idx][1]
        )

        # To get its realtion with Average Grades
        sns.scatterplot(
            x=column, y='Average Grades', data=df,
            color='#706dbd', alpha=.7, s=80,
            ax=axs[idx][2]
        )
        
        
plot_base_relation(df.select_dtypes(include=[int, float]))

Here we won't be removing outliers since the dataset size is small

In [ ]:
def plot_base_categorical_relation(df, figsize=(20, 60)):
    columns = df.columns.tolist()
    _, axs = plt.subplots(len(columns), 3, figsize=figsize)
    
    for idx, column in enumerate(columns):
        try:
            # To get knowledge about outliers & distribution
            sns.boxplot(x=df[column], y=df['Average Grades'], ax=axs[idx][0])

            # To get its realtion with Average Grades
            sns.stripplot(
                x=column, y='Average Grades', data=df,
                color='#706dbd', alpha=.7, jitter=.1,
                ax=axs[idx][1]
            )

            # To get count plot for `column` (considering NaN, so we can know 
            # how much of data is missing)
            plot_countplot(df, column, axs[idx][2])
        except ValueError:
            # ValueError: min() arg is an empty sequence
            # 
            # The above error happens while creating plot for some columns (maybe 
            # because it has NaN value)
            print(f'{column} cannot be plotted')
        
        
plot_base_categorical_relation(
    pd.concat(
        [df.select_dtypes(include=['object']), df[['Average Grades']]],
        axis='columns'
    )
)

Since the number of samples and categories in categorical columns are less, therefore doing `one hot encoding`

In [ ]:
ohe_df = pd.get_dummies(df.select_dtypes('object'))
ohe_df.head()

In [ ]:
# Removing one column after doing one hot encoding to avoid multi-collinearity issues
ohe_df.drop(['romantic_yes'], axis='columns', inplace=True)

In [ ]:
# Removing the categorical columns and adding one hot encoded df

# Removing
categorical_columns = df.select_dtypes('object').columns.tolist()
df.drop(categorical_columns, axis='columns', inplace=True)

# Adding
df = pd.concat([df, ohe_df], axis='columns')
df.head()

In [ ]:
# Since is collinear to G1, G2 & G3
df.drop(['Average Grades'], axis='columns', inplace=True)

## 🎸 Modelling

Creating a regression model that can predict student's final grades.

In [ ]:
# KFold for cross validation
kf = KFold(n_splits=10, shuffle=True)

In [ ]:
# Shuffling the dataset
df = df.sample(frac=1, random_state=5)

In [ ]:
# Selecting features by analysing which features are collinear to `G3` and collinear 
# to the selected columns
features = ['failures', 'Medu', 'studytime', 'absences', 'G1', 'G2', 'higher_no', 'higher_yes']
target = 'G3'

x_train, x_test, y_train, y_test = train_test_split(
    df[features], df[target], test_size=0.3, random_state=0
)

In [ ]:
# Scaling the dataset

scaler = StandardScaler()

x_train = scaler.fit_transform(np.asanyarray(x_train))
y_train = np.asanyarray(y_train)

x_test = scaler.fit_transform(np.asanyarray(x_test))
y_test = np.asanyarray(y_test)

In [ ]:
# Cross Validation
scoring = 'r2'
score = cross_val_score(linear_model.LinearRegression(), x_train, y_train, cv=4, scoring=scoring)
score.mean()

In [ ]:
# Plotting learning curve
_sizes = [i for i in range(1, 408, 10)]
train_sizes = np.array([_sizes])  # Relative sizes
scoring = 'neg_mean_squared_error'

lr = linear_model.LinearRegression()
train_sizes_abs, train_scores, cv_scores = learning_curve(
    lr, x_train, y_train, train_sizes=train_sizes, cv=10, scoring=scoring
)

In [ ]:
train_scores_mean = []
for row in train_scores:
    _mean = row.mean()
    train_scores_mean.append(_mean)
    
cv_scores_mean = []
for row in cv_scores:
    _mean = row.mean()
    cv_scores_mean.append(_mean)    
    
train_scores_mean = -np.array(train_scores_mean)
cv_scores_mean = -np.array(cv_scores_mean)
    
print(train_scores_mean)
print()
print(cv_scores_mean)

In [ ]:
plt.plot(train_sizes_abs, train_scores_mean, label='Train')
plt.plot(train_sizes_abs, cv_scores_mean, label='Cross Validation')

plt.legend()

In [ ]:
# Fitting the model
model = lr.fit(x_train, y_train)

In [ ]:
# Optimal parameter
coefficients = model.coef_
intercept = model.intercept_

print("Coefficients: ", coefficients)
print("Intercept: ", model.intercept_)

## ⛱ Evaluation

In [ ]:
y_test_pred = model.predict(x_test)

In [ ]:
# To see how our model performs on data that model has NOT seen

rms_error = mean_squared_error(y_test, y_test_pred, squared=False)
r2_score_value = r2_score(y_test, y_test_pred)

print(f"Root mean squared error: {rms_error}")
print(f"R2-score: {r2_score_value}")

**Creating a pipeline**

In [ ]:
scaling = ('scale', StandardScaler())
model = ('model', linear_model.LinearRegression())

# Steps in the pipeline
steps = [scaling, model]

pipe = Pipeline(steps=steps)

# Fiitting the model
model = pipe.fit(x_train, y_train)

# Out-Of-Sample Forecast
y_test_pred = model.predict(x_test)

# Evaluation
rms_error = mean_squared_error(y_test, y_test_pred, squared=False)
r2_score_value = r2_score(y_test, y_test_pred)

print(f"Root mean squared error: {rms_error}")
print(f"R2-score: {r2_score_value}")

In [ ]:
# Saving the model
dump(model, 'model.joblib')

`Visualizing` our prediction against actual values.

In [ ]:
f, (ax1, ax2) = plt.subplots(nrows=1, ncols=2, figsize=(13, 5))

ax1.plot(np.arange(len(y_test)), y_test, label='Actual')
ax2.plot(np.arange(len(y_test_pred)), y_test_pred, label='Prediction')

ax1.legend()
ax2.legend()

f, ax3 = plt.subplots(nrows=1, ncols=1, figsize=(13, 5))

ax3.plot(np.arange(len(y_test)), y_test, label='Actual')
ax3.plot(np.arange(len(y_test_pred)), y_test_pred, label='Prediction')

ax3.legend()

---

I'll wrap things up there. If you want to find some other answers then go ahead `edit` this kernel. If you have any `questions` then do let me know.

If this kernel helped you then don't forget to 🔼 `upvote` and share your 🎙 `feedback` on improvements of the kernel.

![](https://media.giphy.com/media/ny7UCd6JETnmE/giphy.gif)

---